In [17]:
import casadi as ca
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.io

from scipy.linalg import solve_continuous_are

from ocslc.switched_linear_mpc import SwiLin
from optimizers.sgd import StochasticGradientDescent, RMSPropOptimizer, AdamOptimizer
from optimizers.sgd import sgd_optimize, rmsprop_optimize, adam_optimize

def switched_problem(n_phases=5):
    """
    Set up a switched linear problem and compute cost and gradient functions
    """
    model = {
        'A': [
            np.array([[-2.5, 0.5, 0.3], [0.4, -2.0, 0.6], [0.2, 0.3, -1.8]]),
            np.array([[-1.9, 3.2, 0.4], [0.3, -2.1, 0.5], [0, 0.6, -2.3]]),
            np.array([[-2.2, 0, 0.5],   [0.2, -1.7, 0.4], [0.3, 0.2, -2.0]]),
            np.array([[-1.8, 0.3, 0.2], [0.5, -2.4, 0],   [0.4, 0, -2.2]]),
            np.array([[-2.0, 0.4, 0],   [0.3, -2.2, 0.2], [0.5, 0.3, -1.9]]),
            np.array([[-2.3, 0.2, 0.3], [0, -2.0, 0.4],   [0.2, 0.5, -2.1]]),
            np.array([[-1.7, 0.5, 0.4], [0.2, -2.5, 0.3], [1.1, 0.2, -2.4]]),
            np.array([[-2.1, 0.3, 0.2], [0.4, -1.9, 0.5], [0.3, 0.1, -2.0]]),
            np.array([[-2.4, 0, 0.5],   [0.2, -2.3, 0.3], [0.4, 0.2, -1.8]]),
            np.array([[-1.8, 0.4, 0.3], [0.5, -2.1, 0.2], [0.2, 3.1, -2.2]]),
        ],
        'B': [
            np.array([[1.5, 0.3], [0.4, 1.2], [0.2, 0.8]]),
            np.array([[1.2, 0.5], [0.3, 0.9], [0.4, 1.1]]),
            np.array([[1.0, 0.4], [0.5, 1.3], [0.3, 0.7]]),
            np.array([[1.4, 0.2], [0.6, 1.0], [0.1, 0.9]]),
            np.array([[1.3, 0.1], [0.2, 1.4], [0.5, 0.6]]),
            np.array([[1.1, 0.3], [0.4, 1.5], [0.2, 0.8]]),
            np.array([[1.6, 0.2], [0.3, 1.1], [0.4, 0.7]]),
            np.array([[1.0, 0.4], [0.5, 1.2], [0.3, 0.9]]),
            np.array([[1.2, 0.5], [0.1, 1.3], [0.6, 0.8]]),
            np.array([[1.4, 0.3], [0.2, 1.0], [0.5, 0.7]]),
        ],
    }

    n_states = model['A'][0].shape[0]
    n_inputs = model['B'][0].shape[1]

    time_horizon = 10

    x0 = np.array([2, -1, 5])
    
    xr = np.array([1, -3])
    
    swi_lin = SwiLin(
        n_phases, 
        n_states,
        n_inputs,
        time_horizon, 
        auto=False, 
    )
    
    # Load model
    swi_lin.load_model(model)

    Q = 10. * np.eye(n_states)
    R = 10. * np.eye(n_inputs)
    E = 1. * np.eye(n_states)

    swi_lin.precompute_matrices(x0, Q, R, E)
    x0 = np.append(x0, 1)  # augment with 1 for affine term
    J_func = swi_lin.cost_function(R, x0)
        
    grad_J_u = []
    grad_J_delta = []

    for k in range(n_phases):
        # Compute gradient of the cost
        du, d_delta = swi_lin.grad_cost_function(k, R)
        # print(f"Length du: {len(du)}")

        grad_J_u += du
        grad_J_delta.append(d_delta)

    grad_J = ca.vertcat(*grad_J_delta, *grad_J_u)

    # keep the original stacked forms if needed
    grad_J_u = np.hstack(grad_J_u)
    grad_J_delta = np.hstack(grad_J_delta)
    
    # Create a CasADi function for the gradient
    # grad_J_func = ca.Function('grad_J', [*swi_lin.u, *swi_lin.delta], [grad_J])
    
    u_vec = ca.vertcat(*swi_lin.u)
    delta_vec = ca.vertcat(*swi_lin.delta)

    grad_J_func = ca.Function('grad_J', [u_vec, delta_vec], [grad_J])
        
    # Create wrapper functions for the optimizer
    def cost_function(params, indices=None, data=None):
        """
        params: 1D array (n_params,) or 2D array (batch_size, n_params)
        indices: optional indices to select a minibatch from a 2D params array
        Returns the scalar loss (averaged over minibatch if batch provided).
        """
        params = np.asarray(params)
        # From a single flattened params vector, unpack into controls and durations
        u = params[:n_phases * n_inputs].reshape((n_phases, n_inputs)).tolist()
        phases_duration = params[n_phases * n_inputs:].reshape((n_phases,)).tolist()
        params_list = u + phases_duration
        # Compute the cost function for a single example (no batch)
        J = float(J_func(*params_list).full().item())
        J = 0
        
        if data is not None:
            u = data['controls'].ravel()
            phases_duration = data['phases_duration'].ravel()
            params_ref = np.concatenate([u, phases_duration])
            # print(f"Reference params: {params_ref}")
            
            # Compute the loss wrt reference params
            params_ref = np.asarray(params_ref)
            # add numpy sum of squared differences to scalar loss
            J += float(np.sum((params - params_ref) ** 2) / len(params_ref))

        return J

    def gradient_function(params, indices=None, data=None):
        """
        params: 1D array (n_params,) or 2D array (batch_size, n_params)
        indices: optional indices to select a minibatch from a 2D params array
        Returns gradient vector (n_params,) averaged over minibatch if batch provided.
        """
        params = np.asarray(params)
        # From a single flattened params vector, unpack into controls and durations
        u = params[:n_phases * n_inputs].reshape((n_phases, n_inputs)).tolist()
        phases_duration = params[n_phases * n_inputs:].reshape((n_phases,)).tolist()
        params_list = u + phases_duration
        # single example
        grad_J = np.asarray(grad_J_func(*params_list).full().ravel())
        grad_J = 0

        if data is not None:
            u = data['controls'].ravel()
            phases_duration = data['phases_duration'].ravel()
            params_ref = np.concatenate([u, phases_duration])
            params_ref = np.asarray(params_ref)
            # add gradient of the squared differences to grad_J
            grad_J = 2 * (params - params_ref) / len(params_ref)

        return grad_J


    return J_func, grad_J_func, cost_function, gradient_function

In [ ]:
# Get the cost and gradient functions from the switched problem
J_func, grad_J_func, cost_function, gradient_function = switched_problem(n_phases=1)

# Define range for u and delta (for single phase problem)
n_phases = 1
n_inputs = 2
u1_values = np.linspace(-5, 5, 50)
u2_values = np.linspace(-5, 5, 50)
delta_values_switch = np.linspace(0.1, 2.0, 50)

# Create 2D plots for cost function with respect to (u1, u2) at fixed delta
U1, U2 = np.meshgrid(u1_values, u2_values)
J_switch = np.zeros_like(U1)
fixed_delta = 1.0

for i in range(len(u2_values)):
    for j in range(len(u1_values)):
        u = [[u1_values[j], u2_values[i]]]
        phases_duration = [fixed_delta]
        J_switch[i, j] = float(J_func(*u, *phases_duration).full().item())

# Create plots
fig = plt.figure(figsize=(15, 5))

# 3D surface plot
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot_surface(U1, U2, J_switch, cmap='viridis')
ax1.set_xlabel('u1')
ax1.set_ylabel('u2')
ax1.set_zlabel('J')
ax1.set_title(f'Switched System Cost J(u1, u2) at δ={fixed_delta}')

# Contour plot
ax2 = fig.add_subplot(132)
contour = ax2.contourf(U1, U2, J_switch, levels=20, cmap='viridis')
ax2.set_xlabel('u1')
ax2.set_ylabel('u2')
ax2.set_title(f'Cost Function J(u1, u2) - Contour at δ={fixed_delta}')
plt.colorbar(contour, ax=ax2)

# Plot cost vs delta at fixed u
ax3 = fig.add_subplot(133)
J_vs_delta = np.zeros(len(delta_values_switch))
fixed_u = [[0.0, 0.0]]
for i, delta in enumerate(delta_values_switch):
    J_vs_delta[i] = float(J_func(*fixed_u, delta).full().item())
ax3.plot(delta_values_switch, J_vs_delta, 'b-', linewidth=2)
ax3.set_xlabel('delta')
ax3.set_ylabel('J')
ax3.set_title('Cost Function J(δ) at u=[0, 0]')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
# plt.show()

# Plot gradients
fig2 = plt.figure(figsize=(15, 10))

# Calculate gradients at fixed delta
grad_u1 = np.zeros_like(U1)
grad_u2 = np.zeros_like(U2)

for i in range(len(u2_values)):
    for j in range(len(u1_values)):
        u = [[u1_values[j], u2_values[i]]]
        phases_duration = [fixed_delta]
        grad = np.array(grad_J_func(*u, *phases_duration).full()).ravel()
        grad_u1[i, j] = grad[1]  # gradient wrt u1
        grad_u2[i, j] = grad[2]  # gradient wrt u2

# Plot gradient wrt u1
ax4 = fig2.add_subplot(221, projection='3d')
ax4.plot_surface(U1, U2, grad_u1, cmap='plasma')
ax4.set_xlabel('u1')
ax4.set_ylabel('u2')
ax4.set_zlabel('∂J/∂u1')
ax4.set_title(f'Gradient ∂J/∂u1 at δ={fixed_delta}')

# Plot gradient wrt u2
ax5 = fig2.add_subplot(222, projection='3d')
ax5.plot_surface(U1, U2, grad_u2, cmap='coolwarm')
ax5.set_xlabel('u1')
ax5.set_ylabel('u2')
ax5.set_zlabel('∂J/∂u2')
ax5.set_title(f'Gradient ∂J/∂u2 at δ={fixed_delta}')

# Contour plots
ax6 = fig2.add_subplot(223)
contour1 = ax6.contourf(U1, U2, grad_u1, levels=20, cmap='plasma')
ax6.set_xlabel('u1')
ax6.set_ylabel('u2')
ax6.set_title(f'Gradient ∂J/∂u1 - Contour at δ={fixed_delta}')
plt.colorbar(contour1, ax=ax6)

ax7 = fig2.add_subplot(224)
contour2 = ax7.contourf(U1, U2, grad_u2, levels=20, cmap='coolwarm')
ax7.set_xlabel('u1')
ax7.set_ylabel('u2')
ax7.set_title(f'Gradient ∂J/∂u2 - Contour at δ={fixed_delta}')
plt.colorbar(contour2, ax=ax7)

plt.tight_layout()
# plt.show()

# Plot gradient wrt delta
grad_delta_vs_delta = np.zeros(len(delta_values_switch))
for i, delta in enumerate(delta_values_switch):
    grad = np.array(grad_J_func(*fixed_u, delta).full()).ravel()
    grad_delta_vs_delta[i] = grad[0]  # gradient wrt delta

fig3 = plt.figure(figsize=(10, 5))
ax8 = fig3.add_subplot(111)
ax8.plot(delta_values_switch, grad_delta_vs_delta, 'r-', linewidth=2)
ax8.set_xlabel('delta')
ax8.set_ylabel('∂J/∂δ')
ax8.set_title('Gradient ∂J/∂δ at u=[0, 0]')
ax8.grid(True, alpha=0.3)
plt.tight_layout()
# plt.show()

In [ ]:
# Analyze gradient discontinuities
fig_disc = plt.figure(figsize=(18, 12))

# Calculate gradient magnitude
grad_magnitude = np.sqrt(grad_u1**2 + grad_u2**2)

# Calculate spatial derivatives to find discontinuities
# Using central differences to detect abrupt changes
du1_du1 = np.gradient(grad_u1, axis=1)
du1_du2 = np.gradient(grad_u1, axis=0)
du2_du1 = np.gradient(grad_u2, axis=1)
du2_du2 = np.gradient(grad_u2, axis=0)

# Total variation measure (high values indicate discontinuities)
total_variation_u1 = np.abs(du1_du1) + np.abs(du1_du2)
total_variation_u2 = np.abs(du2_du1) + np.abs(du2_du2)
total_variation = total_variation_u1 + total_variation_u2

# Plot 1: Gradient magnitude
ax1_disc = fig_disc.add_subplot(231)
im1 = ax1_disc.contourf(U1, U2, grad_magnitude, levels=30, cmap='viridis')
ax1_disc.set_xlabel('u1')
ax1_disc.set_ylabel('u2')
ax1_disc.set_title('Gradient Magnitude')
plt.colorbar(im1, ax=ax1_disc)

# Plot 2: Total variation (discontinuity indicator)
ax2_disc = fig_disc.add_subplot(232)
im2 = ax2_disc.contourf(U1, U2, total_variation, levels=30, cmap='hot')
ax2_disc.set_xlabel('u1')
ax2_disc.set_ylabel('u2')
ax2_disc.set_title('Total Variation (Discontinuity Indicator)')
plt.colorbar(im2, ax=ax2_disc)

# Plot 3: Highlight high discontinuity regions
ax3_disc = fig_disc.add_subplot(233)
threshold = np.percentile(total_variation, 90)  # Top 10% variations
discontinuity_mask = total_variation > threshold
ax3_disc.contourf(U1, U2, discontinuity_mask.astype(float), levels=[0, 0.5, 1], cmap='RdYlGn_r')
ax3_disc.set_xlabel('u1')
ax3_disc.set_ylabel('u2')
ax3_disc.set_title(f'High Discontinuity Regions (>{threshold:.2f})')

# Plot 4: Second derivative magnitude for ∂J/∂u1
ax4_disc = fig_disc.add_subplot(234)
im4 = ax4_disc.contourf(U1, U2, total_variation_u1, levels=30, cmap='plasma')
ax4_disc.set_xlabel('u1')
ax4_disc.set_ylabel('u2')
ax4_disc.set_title('Variation in ∂J/∂u1')
plt.colorbar(im4, ax=ax4_disc)

# Plot 5: Second derivative magnitude for ∂J/∂u2
ax5_disc = fig_disc.add_subplot(235)
im5 = ax5_disc.contourf(U1, U2, total_variation_u2, levels=30, cmap='coolwarm')
ax5_disc.set_xlabel('u1')
ax5_disc.set_ylabel('u2')
ax5_disc.set_title('Variation in ∂J/∂u2')
plt.colorbar(im5, ax=ax5_disc)

# Plot 6: Line plot showing gradient behavior along u1 at u2=0
ax6_disc = fig_disc.add_subplot(236)
u2_zero_idx = np.argmin(np.abs(u2_values))
ax6_disc.plot(u1_values, grad_u1[u2_zero_idx, :], 'b-', label='∂J/∂u1', linewidth=2)
ax6_disc.plot(u1_values, grad_u2[u2_zero_idx, :], 'r-', label='∂J/∂u2', linewidth=2)
ax6_disc.axhline(y=0, color='k', linestyle='--', alpha=0.3)
ax6_disc.axvline(x=0, color='k', linestyle='--', alpha=0.3)
ax6_disc.set_xlabel('u1')
ax6_disc.set_ylabel('Gradient')
ax6_disc.set_title('Gradient Components at u2≈0')
ax6_disc.legend()
ax6_disc.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print statistics about discontinuities
print("Discontinuity Analysis:")
print(f"Total variation range: [{total_variation.min():.4f}, {total_variation.max():.4f}]")
print(f"Mean total variation: {total_variation.mean():.4f}")
print(f"90th percentile threshold: {threshold:.4f}")
print(f"\nPercentage of points with high variation: {100 * np.sum(discontinuity_mask) / discontinuity_mask.size:.2f}%")

# Find regions with maximum discontinuity
max_disc_idx = np.unravel_index(np.argmax(total_variation), total_variation.shape)
print(f"\nMaximum discontinuity at:")
print(f"  u1 = {U1[max_disc_idx]:.4f}")
print(f"  u2 = {U2[max_disc_idx]:.4f}")
print(f"  Total variation = {total_variation[max_disc_idx]:.4f}")

In [ ]:
def delta_t(theta, T, n, epsilon):
    """
    Compute Δt_k = ε + (T - nε) * exp(θ_k) / sum_j exp(θ_j)
    
    Parameters
    ----------
    theta : array_like
        Array of θ_k values.
    T : float
        Total time or scaling parameter.
    n : int
        Number of terms (e.g. len(theta)).
    epsilon : float
        Minimum time step ε.
    
    Returns
    -------
    delta_t : np.ndarray
        Array of Δt_k values.
    """
    theta = np.array(theta)
    softmax = np.exp(theta) / np.sum(np.exp(theta))
    delta_t = epsilon + (T - n * epsilon) * softmax
    return delta_t

# Example usage:
theta = [0.2, 1.0, -0.5]
T = 10.0
epsilon = 0.01
n = len(theta)

dt = delta_t(theta, T, n, epsilon)
print("Δt_k:", dt)
print("Sum of Δt_k =", np.sum(dt))


## Class top wrap a Casadi Function into a Torch object

In [19]:
import torch
import torch.nn as nn
import casadi as ca
import numpy as np

class CasadiFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, casadi_fn, *inputs):
        """
        Generic forward pass for any CasADi function.
        inputs: Sequence of PyTorch tensors matching CasADi arguments.
        """
        # 1. Check dimensions and retrieve batch size
        # We assume the first dimension is the Batch dimension [Batch, Features]
        batch_size = inputs[0].shape[0]
        
        # 2. Prepare inputs for CasADi (Batch, Features) -> (Features, Batch)
        # CasADi .map() expects column-major data
        inputs_np = [x.detach().cpu().numpy().T for x in inputs]
        
        # 3. Run CasADi function
        # .map allows us to run the function over the whole batch at once
        result_dm = casadi_fn.map(batch_size)(*inputs_np)
        
        # 4. Handle output (CasADi might return a single item or a list)
        if isinstance(result_dm, (list, tuple)):
            results_np = [np.array(r).T for r in result_dm]
        else:
            results_np = [np.array(result_dm).T]
            
        # 5. Save context for backward (gradients)
        ctx.casadi_fn = casadi_fn
        ctx.save_for_backward(*inputs)
        ctx.batch_size = batch_size
        ctx.n_outputs = len(results_np)

        # 6. Convert back to torch
        results_torch = [torch.from_numpy(r).float().to(inputs[0].device) for r in results_np]
        
        # If specific function has only 1 output, unpack it
        return results_torch[0] if len(results_torch) == 1 else tuple(results_torch)

    @staticmethod
    def backward(ctx, *grad_outputs):
        """
        Generic backward pass using CasADi AD.
        """
        inputs = ctx.saved_tensors
        casadi_fn = ctx.casadi_fn
        batch_size = ctx.batch_size
        
        # 1. Prepare Inputs and Incoming Gradients (transposed)
        inputs_np = [x.detach().cpu().numpy().T for x in inputs]
        grad_outputs_np = [g.detach().cpu().numpy().T for g in grad_outputs]
        
        # 2. Compute Jacobian-Vector Product (Reverse Mode AD)
        # We assume the user wants gradients w.r.t all inputs
        # We must generate the reverse-mode AD function for THIS specific function structure
        
        # NOTE: For maximum efficiency, this graph generation should be cached in __init__, 
        # but for generic "conversion" we do it here safely.
        
        # Reconstruct symbolic inputs based on numerical shapes
        sym_inputs = [ca.MX.sym(f'in_{i}', inp.shape[0], 1) for i, inp in enumerate(inputs_np)]
        sym_out = casadi_fn(*sym_inputs)
        
        # Ensure sym_out is a list
        if not isinstance(sym_out, (list, tuple)):
            sym_out = [sym_out]

        # Calculate gradients: sum( adj * output )
        # CasADi requires matching the number of output gradients
        adj_inputs = ca.jtimes(ca.vertcat(*sym_out), ca.vertcat(*sym_inputs), ca.vertcat(*grad_outputs_np), True)
        
        # The result of jtimes is one giant vector; we need to split it back into input shapes
        # This split logic can be complex; a simpler approach for a quick converter 
        # is to ask CasADi for gradients one by one or create a specific function.
        
        # --- Optimized Reverse Mode Mapping ---
        # We create a function: f_bwd(inputs, grad_outputs) -> grad_inputs
        bwd_name = f"{casadi_fn.name()}_bwd"
        sym_grads = [ca.MX.sym(f'g_{i}', g.shape[0], 1) for i, g in enumerate(grad_outputs_np)]
        
        # Reverse mode AD
        # gradient of (outputs dot grad_outputs) w.r.t inputs
        out_dot_grad = 0
        for out_node, grad_node in zip(sym_out, sym_grads):
            out_dot_grad += ca.dot(out_node, grad_node)
            
        grads_per_input = ca.gradient(out_dot_grad, ca.vertcat(*sym_inputs))
        
        # Create the backward function
        # Inputs: [*inputs, *grad_outputs] -> Output: [grads_per_input]
        # Note: grads_per_input is a flat vector, might need splitting if inputs were distinct
        # For simplicity in this generic converter, we assume inputs are vector-like enough to be vertcat'd
        
        bwd_fn = ca.Function(bwd_name, [*sym_inputs, *sym_grads], [grads_per_input])
        
        # Execute
        all_grads_dm = bwd_fn.map(batch_size)(*inputs_np, *grad_outputs_np)
        all_grads_np = np.array(all_grads_dm) # Shape: (Total_Input_Dim, Batch)
        
        # Split gradients back to match input tensors
        grad_inputs_torch = []
        idx = 0
        for inp in inputs_np:
            rows = inp.shape[0]
            grad_segment = all_grads_np[idx : idx+rows, :]
            grad_inputs_torch.append(torch.from_numpy(grad_segment.T).float().to(inputs[0].device))
            idx += rows
            
        return (None, *grad_inputs_torch)

class CasadiToTorch(nn.Module):
    def __init__(self, casadi_fn):
        super().__init__()
        self.casadi_fn = casadi_fn
        
    def forward(self, *args):
        # The apply method handles the autograd linking
        return CasadiFunction.apply(self.casadi_fn, *args)

In [20]:
# 1. Your existing CasADi setup
# Get the cost and gradient functions from the switched problem
n_phases = 10
n_inputs = 2
J_func, grad_J_func, cost_function, gradient_function = switched_problem(n_phases=10)

# 2. Convert to Torch Layer
torch_grad_J = CasadiToTorch(grad_J_func)

# 3. Use in PyTorch
# Define batch size
B = 32

# Create dummy tensors matching your specific u and delta dimensions
# Example: if u has 3 elements and delta has 2 elements -> 5 arguments total
# Important: Inputs must be (Batch_Size, Dimensions)
u1_tensor = torch.randn(B, n_inputs*n_phases, requires_grad=True)
u2_tensor = torch.randn(B, n_phases, requires_grad=True)
# ... etc for all inputs in [*u, *delta]
print(f"Input shapes: u1: {u1_tensor.shape}, u2: {u2_tensor.shape}")


# 4. Call it (Forward pass)
output = torch_grad_J(u1_tensor, u2_tensor) # Add all your arguments here

# 5. Use Torch methods
print(f"Output shape: {output.shape}")

loss = output.mean()
loss.backward() # This will differentiate through your CasADi gradient function

print(f"Gradient of first input: {u1_tensor.grad}")

Input shapes: u1: torch.Size([32, 20]), u2: torch.Size([32, 10])
Output shape: torch.Size([32, 30])
Gradient of first input: tensor([[-1.3953e+05, -4.7544e+05, -9.8642e+04,  3.9861e+04,  2.7634e+04,
          3.0189e+05,  5.9914e+04,  3.9670e+04, -2.6782e+04,  1.0059e+04,
          3.3642e+04,  1.3058e+03,  3.5546e+03,  2.1279e+02,  1.4075e+02,
         -3.2698e+02, -1.9437e+02,  2.9531e+02,  3.6018e+00, -1.1251e+00],
        [ 4.1306e+07,  2.2889e+08,  4.3963e+07, -4.8989e+07,  2.1738e+04,
         -3.8379e+05, -9.5960e+04, -4.9556e+05,  3.0802e+05, -2.1140e+05,
          2.0779e+04,  1.3790e+04, -8.9711e+03, -4.9281e+03,  9.5540e+03,
          1.1771e+04, -6.8310e+03, -4.5519e+03, -3.4519e+03, -6.0129e+03],
        [-2.1482e+00, -3.0445e-01, -2.5798e+00,  2.6763e-01,  1.2575e+00,
         -8.9922e+00,  6.9783e-01, -1.7331e-01, -4.3527e-01, -1.9193e+00,
          4.0935e-02, -4.5655e-02,  7.9704e-02, -2.2496e-02,  1.3103e-02,
          1.6451e-02, -9.2621e-02, -6.4626e-02,  1.0451e-02